# Ikeda for many ships
There are some interesting section data in the manoeuvring database, here it is investigated if this can be used as input to *ScoresII*.

In [ ]:
# %load ../../imports.py
"""
These is the standard setup for the notebooks.
"""

%matplotlib inline
%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
#plt.style.use('paper')

#import data
import copy
from rolldecay.bis_system import BisSystem
from rolldecay import database
from mdldb.tables import Run

from sklearn.pipeline import Pipeline
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from rolldecayestimators.direct_estimator_cubic import EstimatorQuadraticB, EstimatorCubic
from rolldecayestimators.ikeda_estimator import IkedaQuadraticEstimator
import rolldecayestimators.equations as equations
import rolldecayestimators.lambdas as lambdas
from rolldecayestimators.substitute_dynamic_symbols import lambdify
import rolldecayestimators.symbols as symbols
import sympy as sp

from sklearn.metrics import r2_score



In [ ]:
df = pd.read_csv('sections.csv', sep=';', index_col=0)
mask=df['b20'].notnull()
df=df.loc[mask].copy()
df.drop(index=['DAE18A','DAE18B'], inplace=True)  # Bad project number
df['Project No']=df['Project No'].astype(int)

In [ ]:
df_rolldecay = database.load(rolldecay_table_name='rolldecay_quadratic_b', limit_score=0.99, 
                             exclude_table_name='rolldecay_exclude')

In [ ]:
interesting_projects = set(df_rolldecay['project_number']) & set(df['Project No'])
interesting_projects

In [ ]:
def assemble_sections(row):
    
    b=np.zeros(21)
    area=np.zeros(21)
    t=np.zeros(21)
    for i in range(21):
        b_key='b%i' % i
        area_key='area%i' % i
        t_key = 't%i' % i
        b[i]=row[b_key]
        area[i]=row[area_key]
        t[i]=row[t_key]
    
    df_sections = pd.DataFrame()
    df_sections['b']=b
    df_sections['area']=area
    df_sections['t']=t
    df_sections['SHIP']=int(row['SHIP'])
    df_sections['Project No']=row['Project No']
    df_sections.dropna(inplace=True)
    return df_sections

In [ ]:
df_sections = pd.DataFrame()

for index, row in df.iterrows():
    df_ = assemble_sections(row=row)
    df_['ID']=index
    df_sections=df_sections.append(df_)

In [ ]:
def count_variations(df_group):
    df_group=df_group.copy()
    df_group.drop(columns=['SHIP'], inplace=True)
    
    for key, values in df_group.items():
        variations = len(values.unique())
        if not variations==1:
            break
            
    return variations
    

In [ ]:
df.groupby(by='Project No').get_group('20000157')

In [ ]:
df.groupby(by='Project No').apply(func=count_variations)            
    
        
    

In [ ]:
df_rolldecay.iloc[0]